In [13]:
# coding:utf-8
from __future__ import print_function
from random import shuffle
from past.builtins import xrange
import pickle
import numpy as np

# 文件读取
def unpickle(file):
	with open(file,'rb') as fo:
		dict = pickle.load(fo)
	return dict

def load_file(file):
	dictTrain = unpickle(file + "data_batch_1")
	dataTrain = dictTrain['data']
	labelTrain = dictTrain['labels']

	for i in range(2,6):
		dictTrain = unpickle(file + "data_batch_" + str(i))
		dataTrain = np.vstack([dataTrain,dictTrain['data']])
		labelTrain = np.hstack([labelTrain,dictTrain['labels']])

	dictTest = unpickle(file + "test_batch")
	dataTest = dictTest['data']
	labelTest = dictTest['labels']
	labelTest = np.array(labelTest)

	return dataTrain, labelTrain, dataTest, labelTest


#softmax loss 函数
def softmax_loss_naive(W, X, y, reg):
	'''
		W:权重矩阵
		X:图片训练集(矩阵)
		y:图片训练集标签(数组)
		reg:正则化强度

		return:
			loss:训练集平均loss值
			dW:梯度矩阵
	'''
	#初始化数据
	loss = 0.0
	dW = np.zeros_like(W)
	num_train = X.shape[0]	#样本数
	num_class = W.shape[1]	#样本类别数

	for i in xrange(num_train):
		score = X[i].dot(W)
		score -= np.max(score)	#提高样本稳定性

		correct_score = score[y[i]]
		exp_sum = np.sum(np.exp(score))
		loss += np.log(exp_sum) - correct_score

		for j in xrange(num_class):
			if (j == y[i]):
				dW[:, j] += np.exp(score[j]) / exp_sum * X[i] - X[i]
			else:
				dW[:, j] += np.exp(score[j]) / exp_sum * X[i]


	loss /= num_train
	loss += 0.5 * reg * np.sum(W*W)

	dW /= num_train
	dW += reg * W

	return loss, dW

#线性分类器
class LinearClassifier(object):
	def __init__(self):
		self.W = None

	def train(self, X, y, step_size = 1e-3, reg = 1e-5, num_iters = 100, batch_size = 200, verbose = True):
		'''
		X:图片训练集(矩阵)
		y:图片训练集标签(数组)
		step_size:学习步进速度
		reg:正则化强度
		num_iters:迭代次数
		batch_size:每次迭代图片样本数
		verbose:是否打印信息

		return:
			loss_history:每次训练loss值
		'''
		num_train, dim = X.shape
		num_classes = np.max(y) + 1
	
		if self.W is None:
			self.W = 0.001 * np.random.randn(dim, num_classes)

		loss_history = []

		for it in xrange(num_iters):
			#从样本中不重复随机采batch_size个样本
			sample_index = np.random.choice(num_train, batch_size, replace=False)

			X_batch = X[sample_index, :]
			y_batch = y[sample_index]

			loss, grad = self.loss(X_batch, y_batch, reg)
			loss_history.append(loss)

			self.W += -step_size * grad

			if (verbose and it %10 == 0):
				print('iteration %d / %d, samples: %d, loss: %f' % (it, num_iters, batch_size, loss))

		return loss_history

	def predict(self, X):
		'''
		X:图片训练集(矩阵)

		return:
			y_pred:标签预测值
		'''
		y_pred = np.zeros(X.shape[1])

		score = X.dot(self.W)
		y_pred = np.argmax(score, axis = 1)

		return y_pred


	def loss(self, X_batch, y_batch, reg):
		'''
		X_batch:图片训练集(矩阵)
		y_batch:图片训练集标签(数组)
		reg:正则化强度

		return:
			loss:训练集平均loss值
			dW:梯度矩阵
		'''
		return softmax_loss_naive(self.W, X_batch, y_batch, reg)

In [14]:
from skimage.feature import local_binary_pattern
from PIL import Image
#开始训练
file_path = './'

dataTrFirst, labelTrain, dataTsFirst, labelTest = load_file(file_path)

dataTr = np.zeros((dataTrFirst.shape[0],32*32))
dataTs = np.zeros((dataTsFirst.shape[0],32*32))


for i in range(dataTrFirst.shape[0] -45000):
    img = dataTrFirst[i].reshape((32,32,3))
    img = Image.fromarray(img)
    res = local_binary_pattern(img.convert('L'),8,2,method='uniform')
    dataTr[i] = res.reshape((1,32*32))
print("训练集加载完成")

for i in range(dataTsFirst.shape[0] -1):
    img = dataTsFirst[i].reshape((32,32,3))
    img = Image.fromarray(img)
    res = local_binary_pattern(img.convert('L'),8,2,method='uniform')
    dataTs[i] = res.reshape((1,32*32))
print("测试集加载完成")

dataTrain = dataTrain - np.mean(dataTrain, axis=0)

LC = LinearClassifier()

print('start training ...')
#train(self, X, y, step_size = 1e-3, reg = 1e-5, num_iters = 100, batch_size = 200, verbose = True)
#在dataTrain中不重复随机抽取batch_size个样本，迭代训练num_iters次
loss_all = LC.train(dataTrain, labelTrain, num_iters = 8000, batch_size = 200)


start training ...
iteration 0 / 8000, samples: 200, loss: 9.808217
iteration 10 / 8000, samples: 200, loss: 10605.763411
iteration 20 / 8000, samples: 200, loss: 9304.199885
iteration 30 / 8000, samples: 200, loss: 19357.470584
iteration 40 / 8000, samples: 200, loss: 16554.449563
iteration 50 / 8000, samples: 200, loss: 13248.801083
iteration 60 / 8000, samples: 200, loss: 10138.387817
iteration 70 / 8000, samples: 200, loss: 10918.990310
iteration 80 / 8000, samples: 200, loss: 11331.581816
iteration 90 / 8000, samples: 200, loss: 11071.385903
iteration 100 / 8000, samples: 200, loss: 9529.599919
iteration 110 / 8000, samples: 200, loss: 9089.952142
iteration 120 / 8000, samples: 200, loss: 7366.002454
iteration 130 / 8000, samples: 200, loss: 9860.515369
iteration 140 / 8000, samples: 200, loss: 11059.625637
iteration 150 / 8000, samples: 200, loss: 7284.592109
iteration 160 / 8000, samples: 200, loss: 7384.702900
iteration 170 / 8000, samples: 200, loss: 10297.431417
iteration 180

iteration 1510 / 8000, samples: 200, loss: 5623.901465
iteration 1520 / 8000, samples: 200, loss: 4787.045802
iteration 1530 / 8000, samples: 200, loss: 7464.028349
iteration 1540 / 8000, samples: 200, loss: 6361.864695
iteration 1550 / 8000, samples: 200, loss: 6162.751724
iteration 1560 / 8000, samples: 200, loss: 4658.702686
iteration 1570 / 8000, samples: 200, loss: 4390.125049
iteration 1580 / 8000, samples: 200, loss: 7231.073791
iteration 1590 / 8000, samples: 200, loss: 5856.447442
iteration 1600 / 8000, samples: 200, loss: 8388.256449
iteration 1610 / 8000, samples: 200, loss: 8684.496906
iteration 1620 / 8000, samples: 200, loss: 5143.354317
iteration 1630 / 8000, samples: 200, loss: 4439.868837
iteration 1640 / 8000, samples: 200, loss: 5612.203192
iteration 1650 / 8000, samples: 200, loss: 4151.878924
iteration 1660 / 8000, samples: 200, loss: 3483.072186
iteration 1670 / 8000, samples: 200, loss: 4831.612373
iteration 1680 / 8000, samples: 200, loss: 5696.018566
iteration 

iteration 3000 / 8000, samples: 200, loss: 6902.769420
iteration 3010 / 8000, samples: 200, loss: 5546.401474
iteration 3020 / 8000, samples: 200, loss: 4402.681200
iteration 3030 / 8000, samples: 200, loss: 6695.867150
iteration 3040 / 8000, samples: 200, loss: 5392.888579
iteration 3050 / 8000, samples: 200, loss: 5185.743481
iteration 3060 / 8000, samples: 200, loss: 5762.328866
iteration 3070 / 8000, samples: 200, loss: 8852.478527
iteration 3080 / 8000, samples: 200, loss: 5267.851889
iteration 3090 / 8000, samples: 200, loss: 8897.619818
iteration 3100 / 8000, samples: 200, loss: 7287.597613
iteration 3110 / 8000, samples: 200, loss: 4689.460758
iteration 3120 / 8000, samples: 200, loss: 5959.293745
iteration 3130 / 8000, samples: 200, loss: 7201.109522
iteration 3140 / 8000, samples: 200, loss: 7423.420253
iteration 3150 / 8000, samples: 200, loss: 4023.460111
iteration 3160 / 8000, samples: 200, loss: 5711.269175
iteration 3170 / 8000, samples: 200, loss: 4586.373140
iteration 

iteration 4490 / 8000, samples: 200, loss: 4834.954044
iteration 4500 / 8000, samples: 200, loss: 5923.936903
iteration 4510 / 8000, samples: 200, loss: 6386.132531
iteration 4520 / 8000, samples: 200, loss: 6607.929668
iteration 4530 / 8000, samples: 200, loss: 6549.451319
iteration 4540 / 8000, samples: 200, loss: 6563.691820
iteration 4550 / 8000, samples: 200, loss: 5193.809062
iteration 4560 / 8000, samples: 200, loss: 7614.169309
iteration 4570 / 8000, samples: 200, loss: 6303.885803
iteration 4580 / 8000, samples: 200, loss: 6012.680944
iteration 4590 / 8000, samples: 200, loss: 4727.219086
iteration 4600 / 8000, samples: 200, loss: 6174.627290
iteration 4610 / 8000, samples: 200, loss: 4930.535576
iteration 4620 / 8000, samples: 200, loss: 6981.769405
iteration 4630 / 8000, samples: 200, loss: 7406.536716
iteration 4640 / 8000, samples: 200, loss: 7493.838805
iteration 4650 / 8000, samples: 200, loss: 5689.701774
iteration 4660 / 8000, samples: 200, loss: 5853.660563
iteration 

iteration 5980 / 8000, samples: 200, loss: 5921.212658
iteration 5990 / 8000, samples: 200, loss: 5810.316665
iteration 6000 / 8000, samples: 200, loss: 4594.197560
iteration 6010 / 8000, samples: 200, loss: 4581.494761
iteration 6020 / 8000, samples: 200, loss: 6653.562217
iteration 6030 / 8000, samples: 200, loss: 4140.277410
iteration 6040 / 8000, samples: 200, loss: 7241.776221
iteration 6050 / 8000, samples: 200, loss: 3817.651921
iteration 6060 / 8000, samples: 200, loss: 5394.933146
iteration 6070 / 8000, samples: 200, loss: 3543.612058
iteration 6080 / 8000, samples: 200, loss: 6062.724759
iteration 6090 / 8000, samples: 200, loss: 5859.096476
iteration 6100 / 8000, samples: 200, loss: 3887.003758
iteration 6110 / 8000, samples: 200, loss: 3978.432958
iteration 6120 / 8000, samples: 200, loss: 6533.233415
iteration 6130 / 8000, samples: 200, loss: 6406.341141
iteration 6140 / 8000, samples: 200, loss: 6705.314074
iteration 6150 / 8000, samples: 200, loss: 5707.394052
iteration 

iteration 7470 / 8000, samples: 200, loss: 4883.571387
iteration 7480 / 8000, samples: 200, loss: 5457.763132
iteration 7490 / 8000, samples: 200, loss: 5834.362031
iteration 7500 / 8000, samples: 200, loss: 4644.752470
iteration 7510 / 8000, samples: 200, loss: 6061.524176
iteration 7520 / 8000, samples: 200, loss: 5105.913802
iteration 7530 / 8000, samples: 200, loss: 6709.325092
iteration 7540 / 8000, samples: 200, loss: 4264.911792
iteration 7550 / 8000, samples: 200, loss: 4387.266900
iteration 7560 / 8000, samples: 200, loss: 5596.831950
iteration 7570 / 8000, samples: 200, loss: 6179.258345
iteration 7580 / 8000, samples: 200, loss: 5989.904380
iteration 7590 / 8000, samples: 200, loss: 4271.106230
iteration 7600 / 8000, samples: 200, loss: 4363.402325
iteration 7610 / 8000, samples: 200, loss: 4952.695210
iteration 7620 / 8000, samples: 200, loss: 5157.540779
iteration 7630 / 8000, samples: 200, loss: 5451.920061
iteration 7640 / 8000, samples: 200, loss: 3282.001379
iteration 

In [15]:
print('last loss is %f' %(loss_all[-1]))
#开始预测
print('start predicting ...')
y_pred = LC.predict(dataTest)

hit = 0
for i in xrange(y_pred.size):
	if (y_pred[i] == labelTest[i]):
		hit += 1

print('the accuracy rate is %f ' % (hit/100))

last loss is 5050.960131
start predicting ...
the accuracy rate is 25.000000 
